In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ppo import PPO
from config_utils import get_config
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [3]:
from IPython import display
import gym
from gym import wrappers
env = gym.make("CarRacing-v0")

observation = env.reset()
'''
for _ in range(100):
  plt.imshow(env.render(mode='rgb_array'))
  display.display(plt.gcf()) 
  display.clear_output(wait=True)
  # env.render()
  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action)

  if done:
        break
    #observation = env.reset()
env.close()
'''

/home/gmeyerowitz/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1317..1650 -> 333-tiles track


"\nfor _ in range(100):\n  plt.imshow(env.render(mode='rgb_array'))\n  display.display(plt.gcf()) \n  display.clear_output(wait=True)\n  # env.render()\n  action = env.action_space.sample() # your agent here (this takes random actions)\n  observation, reward, done, info = env.step(action)\n\n  if done:\n        break\n    #observation = env.reset()\nenv.close()\n"

In [4]:
print(env.action_space)
print(type(env.action_space))
print(isinstance(env.action_space, gym.spaces.discrete.Discrete))

print(env.observation_space)
print(type(env.observation_space))
print(isinstance(env.action_space, gym.spaces.discrete.Discrete))

Box(3,)
<class 'gym.spaces.box.Box'>
False
Box(96, 96, 3)
<class 'gym.spaces.box.Box'>
False


In [5]:
config = get_config("configs/config-2020_06_02.json")
# config = get_config("configs/config-default_mujoco.json")
print(config)

num_epochs = 1200
# num_epochs = 100
# num_epochs = 300
minibatch_size = 1

if config.has_training:
    num_epochs = config.training.get("num_epochs", num_epochs)
    minibatch_size = config.training.get("minibatch_size", minibatch_size)

{'ppo_hyperparameters': {'epsilon': 0.2, 'discount_factor': 0.9, 'delta': 0.95, 'actor_critic': {'actor': {'learning_rate': 0.005}, 'critic': {'learning_rate': 0.005, 'alpha': 0.75}}}}


In [12]:
from actor_critics import carracer_ac
actor = carracer_ac.Actor(env)
critic = carracer_ac.Critic(env)

ppo = PPO(
    env,
    config=config,
    actor=actor,
    critic=critic,
    no_cuda=True
)

In [17]:
loss_vals = []
pbar = tqdm(range(num_epochs))
for _ in pbar:
    #for _ in tqdm(range(minibatch_size), desc="minibatch"):
    step_loss_vals = ppo.step()
    loss_vals.append(step_loss_vals[-1])
    pbar.set_description("Loss = {:.6f}".format(loss_vals[-1]))

Track generation: 1044..1310 -> 266-tiles track


IndexError: too many indices for array

In [ ]:
plt.plot(loss_vals)
plt.xlabel("step")
plt.ylabel("loss")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
#env_new = wrappers.Monitor(env, './video/', force = True)
times = 0
env_new = env
observation = env_new.reset()
observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
for _ in range(100):
    env_new.render()
    action = torch.argmax(ppo.pi(observation.float())) 
    observation, reward, done, info = env_new.step(action.cpu().detach().numpy())
    observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
    if not done:
        times += 1
        
print(times)
env_new.close()

In [ ]:

env_new.close()

In [ ]:
import matplotlib.pyplot as plt
import gym
from IPython import display
%matplotlib inline

env = gym.make('CarRacing-v0')
env.reset()

for i in range(100):
   plt.imshow(env.render(mode='rgb_array'))
   display.display(plt.gcf())    
   display.clear_output(wait=True)
   env.step(env.action_space.sample()) # take a random action

env.close()

In [ ]:
from actor_critics import carracer_ac
#act_size = env.action_space.size
#print(act_size)
actor = carracer_ac.Actor(env)